<a href="https://colab.research.google.com/github/endorgobio/IntroduccionAnaliticaPrescriptiva/blob/main/M1C10_ComparacionMetodos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p style="text-align: center;">
    <img alt="banner" height="230px" width="100%" src="https://github.com/endorgobio/IntroduccionAnaliticaPrescriptiva/blob/6cc6029c276aacdf228dcec4796b7b3184cfb8b7/src/header.png?raw=true" hspace="10px" vspace="0px">
</p>



En esta sesión, presentamos una solución al problema de programación de pedidos utilizando un modelo de optimización matemática. De momento, no ahondaremos en la estructura del modelo y sus principales componentes. Sin embargo, en este notebook encontrarás los recursos necesarios para experimentar con el modelo y explorar en detalle su implementación. ¡Una excelente oportunidad para aplicar teoría y práctica en un enfoque matemático riguroso!

# <font color='FD6E72'> **Nuestro primer modelo** </font>

Una compañía de alimentos tiene una única línea de producción que procesa un único producto. El plan de demanda semanal contiene las órdenes de los clientes (en miles de unidades) y la utilidad (en miles de dolares) que cada una de ellas genera.

Las órdenes no pueden fraccionarse, por lo que cada una de ellas se produce completa o no se produce. Dado que la usualmente la capacidad de la línea no es suficiente para satisfacer todas las ordenes, debe decidirse qué producir

|        | Orden 1 | Orden 2 | Orden 3 | Orden 4 | Orden 5 | Orden 6 | Orden 7 | Orden 8 | Orden 9 | Orden 10 |
|--------|---------|---------|---------|---------|---------|---------|---------|---------|---------|----------|
| Cantidad |   16    |   29    |   24    |   20    |   17    |   30    |   32    |   28    |   20    |    20    |
| Utilidad  |   73    |   85    |   89    |   73    |   52    |   71    |   102    |   73    |   93    |    79    |


> **¿Cuáles de las ordenes deben producirse?**





## <font color='#ff6d33'> **Una primera solución** </font>

Lo primero que podemos hacer es pensar en una solución basda en el sentido en nuestra intuición.


<font color='85a900'>**Pregunta:** </font> ¿Cómo construiria una solución para el problema planteado?






Este tipo de soluciones se denominan usualmente **heurísticas**. Proveen soluciones rápidas y en algunos casos de calidad aceptable para el problema planteado. Las heuristicas tienen entre otras las siguientes características:


* Encuentran una solución de forma rápida
* Depende del conocieminto del problema que se resuelve
* Por lo general, no tienen garantia de encontrar la mejor solución



El problema descrito es un caso particular del problema de la mochila (*knapsack problem*) que ha recibido mucha atención en la literatura.

### <font color='85a900'>**Heurística voraz (`greedy`)** </font>

1. **Calcular el Ratio Utilidad/Cantidad:**

  Calcula el ratio de valor a peso para cada pedido. Este ratio indica cuánta utilidad proporciona un pedido por cada unidad demandada.


2. **Ordenar los pedidos por Ratio:**
   
   Ordena los pedidos en orden descendente basado en su ratio Utilidad/Cantidad. El objetivo es priorizar los pedidos que ofrecen el mayor utilidad por la menor cantidad.


3. **Seleccionar pedidos:**
   
   Itera a través de los pedidos en el orden de sus ratios ordenados Para cada pedido, verifica si agregar la cantidad del pedido a la producción total actual excedería la capacidad de la línea.
   * Si no excede la capacidad, agrega el pedido a la lista de pedidos seleccionados.
   * Actualiza el utilidad total y la producción total para incluir el pedido recién seleccionado.


4. **Devolver Resultados:**
  
  Después de procesar todos los pedidos o copar la capacidad de producción máxima, devuelve la lista de pedidos seleccionados, la utilidad total y la cantidad totalproducida.

Aplicando esta heuristica el problema propuesto obtendriamos primero la tabla con las razones de utilidad sobre la cantidad



|            | Orden 1 | Orden 2 | Orden 3 | Orden 4 | Orden 5 | Orden 6 | Orden 7 | Orden 8 | Orden 9 | Orden 10 |
|------------|---------|---------|---------|---------|---------|---------|---------|---------|---------|----------|
| Cantidad   |   16    |   29    |   24    |   20    |   17    |   30    |   32    |   28    |   20    |    20    |
| Utilidad   |   73    |   85    |   89    |   73    |   52    |   71    |   102    |   73    |   93    |    79    |
| Razón (U/C)|  4.56   |  2.93   |  3.71   |  3.65   |  3.06   |  2.37   |  3.19   |  2.61   |  4.65   |   3.95   |

Con esto obtenemos el orden en el cual debe evaluarse la producción de los items



> ```python
[9, 1, 10, 3, 4, 7, 5, 8, 2, 6]
```





Siguiendo los pasos para decidir cuando producir o no un pedido tendriamos:
* se producen los pedidos `[9, 1, 10, 3, 4]`
* La utilidad total es: `407`
* La capacidad usada es: `100`



### <font color='85a900'>**Implementación de la heurística** </font>

La siguiente función implementa el algoritmo heurístico propuesto para dar solución al problema

In [1]:
import numpy as np

def knapsack_greedy(weights, values, capacity):
    """
    Solve the knapsack problem using a greedy approach based on value-to-weight ratio.

    Parameters:
        weights (np.ndarray): Array of item weights.
        values (np.ndarray): Array of item values.
        capacity (int): Maximum capacity of the knapsack.

    Returns:
        selected_items (list): List of indices of selected items.
        total_value (int): Total value of selected items.
        total_weight (int): Total weight of selected items.
    """
    # Calculate value-to-weight ratio for each item
    ratios = values / weights

    # Get indices that would sort the items based on ratio in ascending order
    sorted_indices = np.argsort(ratios)[::-1]

    total_value = 0
    total_weight = 0
    selected_items = []

    # Select items based on sorted ratios
    for i in sorted_indices:
        if total_weight + weights[i] <= capacity:
            selected_items.append(i)
            total_value += values[i]
            total_weight += weights[i]

    selected_items = [item+1 for item in selected_items]
    return selected_items, total_value, total_weight




Empleemos la función para resolver la instancia del problema que se presenta en el caso de estudio

In [2]:
# Example usage
weights = np.array([16, 29, 24, 20, 17, 30, 32, 28, 20, 20])
values = np.array([73, 85, 89, 73, 52, 71, 102, 73, 93, 79])
capacity = 110

selected_items, total_value, total_weight = knapsack_greedy(weights, values, capacity)

print("Items seleccionados:", selected_items)
print("Utilidad total:", total_value)
print("Capacidad usada:", total_weight)

Items seleccionados: [9, 1, 10, 3, 4]
Utilidad total: 407
Capacidad usada: 100


## <font color='#ff6d33'> **Modelo matemático** </font>

### <font color='85a900'> **Formulación** </font>

Es posible modelar este problema mediante expresiones mátematicas:

Primero consideremos las decisiones, Para cada pedido $i$ debe decidirse si se produce o no:

> $x_i =
\begin{cases}
1 & \text{se produce el pedido } i , \\
0 & \text{no se produce el pedido } i
\end{cases}$

Nuestro objetivo es máximizar la utilidad
> $\text{Maximizar} \quad  Z = 73x_1 + 85x_2 + 89x_3 + 73x_4 + 52x_5 + 71x_6 + 51x_7 + 73x_8 + 93x_9 + 79x_{10}$

No debe sobrepasarse la capacidad de la linea
> $16x_1 + 29x_2 + 24x_3 + 20x_4 + 17x_5 + 30x_6 + 32x_7 + 28x_8 + 20x_9 + 20x_{10} \leq 110$



En una forma más compacta (que de momento no detallaremos) podria escribirse como:



> $x_i =
\begin{cases}
1 & \text{se produce el pedido } i , \\
0 & \text{no se produce el pedido } i
\end{cases}$

\begin{align}
    \text{Maximizar} \quad & Z = \sum_{i \in I} v_i x_i \\
    \text{Subjeto a}\\
    & \sum_{i \in I} w_i x_i \leq C \\
    & x_i \in \{0, 1\} \quad \forall i \in I
\end{align}

### <font color='85a900'> **Implementación** </font>

El modelo se implementa en un **lenguaje de modelación** y se resuelve haciendo uso de un **optimizador**. En este caso usaremos `pyomo` y `highspy`, respectivamente

In [3]:
!pip install pyomo
!pip install highspy
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.7 MB/s eta 0:00:00


La siguiente función crea el modelo matemático descrito anteriormente

In [4]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory, Binary, sum_product

def create_knapsack_model(weights, values, capacity):
    """
    Create and solve a knapsack problem model using Pyomo.

    Parameters:
        weights (list): List of item weights.
        values (list): List of item values.
        capacity (int): Maximum capacity of the knapsack.

    Returns:
        model (ConcreteModel): The Pyomo model.
        results (SolverResults): The results from the solver.
    """
    # Create a concrete model
    model = ConcreteModel()

    # Number of items
    n = len(weights)

    # Define sets
    model.I = range(n)

    # Define variables
    model.x = Var(model.I, domain=Binary)

    # Objective: Maximize the total value
    model.obj = Objective(
        expr=sum(model.x[i] * values[i] for i in model.I),
        sense='maximize'
    )

    # Constraint: Total weight should not exceed the capacity
    model.capacity_constraint = Constraint(
        expr=sum(model.x[i] * weights[i] for i in model.I) <= capacity
    )

    # Create a solver
    solver = SolverFactory('appsi_highs')
    results = solver.solve(model, tee=False)

    return model, results




Resolvamos la instancia con los datos que venomos considerando

In [5]:
# Example usage

weights = np.array([16, 29, 24, 20, 17, 30, 32, 28, 20, 20])
values = np.array([73, 85, 89, 73, 52, 71, 102, 73, 93, 79])
capacity = 110

model, results = create_knapsack_model(weights, values, capacity)

# Display results
selected_items = [(i) for i in model.I if model.x[i].value == 1]
capacidad_utilizada = sum(weights[i] for i in selected_items)

print("Utilidad total:", model.obj())
print("Items seleccionados:", selected_items)
print("Capacidad utilizada:", capacidad_utilizada)

Utilidad total: 420.0
Items seleccionados: [0, 3, 6, 8, 9]
Capacidad utilizada: 108


## <font color='#ff6d33'> **Análisis de la solución** </font>

Ahora creamos una función que genere instancias aleatorias de este mismo problema

In [6]:
import numpy as np

import numpy as np
np.random.seed(42)

def generate_knapsack_instance(n_items, min_weight, max_weight, min_value, max_value, capacity_ratio):
    """
    Generate a random instance for the knapsack problem.

    Parameters:
        n_items (int): Number of items.
        min_weight (int): Minimum possible weight of an item.
        max_weight (int): Maximum possible weight of an item.
        min_value (int): Minimum possible value of an item.
        max_value (int): Maximum possible value of an item.
        capacity_ratio (float): Ratio of total weight sum to knapsack capacity (0 < capacity_ratio <= 1).

    Returns:
        weights (np.ndarray): Array of item weights.
        values (np.ndarray): Array of item values.
        capacity (int): Knapsack capacity.
    """
    # Generate random weights and values for the items within the given range
    weights = np.random.randint(min_weight, max_weight + 1, size=n_items)
    values = np.random.randint(min_value, max_value + 1, size=n_items)

    # Define the knapsack capacity as a fraction of the sum of the weights
    capacity = int(np.sum(weights) * capacity_ratio)

    return weights, values, capacity

# Example usage
n_items = 10
min_weight = 10
max_weight = 30
min_value = 50
max_value = 100
capacity_ratio = 0.5

weights, values, capacity = generate_knapsack_instance(n_items, min_weight, max_weight, min_value, max_value, capacity_ratio)

print("Weights:", weights)
print("Values:", values)
print("Capacity:", capacity)



Weights: [16 29 24 20 17 30 16 28 20 20]
Values: [73 85 89 73 52 71 51 73 93 79]
Capacity: 110


Generemos `200` replicas del problema y comparemos que tan buena es la solución que provee el heuristico respecto al modelo exacto. Para ello calculamos la diferencis porcentual entre las soluciones de cada método, denominada `gap`:

> $gap = 100*\dfrac{opt-heur}{opt}$

In [7]:
import numpy as np
import pandas as pd
np.random.seed(42)

min_weight = 10
max_weight = 100
min_value = 50
max_value = 200

n_replicas = 200

data  = []

for n_iter in range(n_replicas):
  n_items = np.random.choice([10, 50, 100, 250, 500])
  capacity_ratio = np.random.uniform(0.2, 0.7)
  weights, values, capacity = generate_knapsack_instance(n_items, min_weight, max_weight, min_value, max_value, capacity_ratio)
  selected_items, total_value, total_weight = knapsack_greedy(weights, values, capacity)
  model, results = create_knapsack_model(weights, values, capacity)# Display results
  optimal_value = model.obj()

  data.append([n_items, total_value, optimal_value])

df = pd.DataFrame(data, columns=['n_items', 'total_value', 'optimal_value'])
df['gap'] = ( df['optimal_value']-df['total_value']) / df['optimal_value'] * 100
df['name']='gap'
df

,n_items,total_value,optimal_value,gap,name
0,250,27093,27140.0,0.173176,gap
1,10,863,920.0,6.195652,gap
2,50,4946,4946.0,0.000000,gap
3,50,3635,3668.0,0.899673,gap
4,50,4658,4692.0,0.724638,gap
...,...,...,...,...,...
195,250,21354,21357.0,0.014047,gap
196,250,19048,19048.0,0.000000,gap
197,500,30596,30597.0,0.003268,gap
198,10,467,471.0,0.849257,gap


veamos gráficamente las diferencias

In [14]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()

legend_order = df['n_items'].unique().tolist()
legend_order = sorted(legend_order)

# Create a strip plot
strip = px.strip(df,
         x='name',
         y='gap',
         color='n_items',
         category_orders={'n_items': legend_order},
         stripmode='overlay')

fig.add_trace(go.Box(x=df['name'],
                     y=df['gap'],
                     fillcolor='rgba(128, 128, 128, 0.2)',  # Grey color with transparency
                      line=dict(color='rgba(128, 128, 128, 0.8)'),  # Grey color with different opacity for the border
                     showlegend=False))
for i in range(len(strip.data)):
  fig.add_trace(strip.data[i])

fig.update_layout(autosize=False,
                  title = "GAP heurístico vs modelo exacto para cada tamaño de instancia",
                  width=600,
                  height=600,
                  legend={'traceorder':'normal'})

fig.show()

## <font color='#ff6d33'> **Conclusiones** </font>

---
**¿Qué es posible concluir de los resultados obeservados?**


---



